<a href="https://colab.research.google.com/github/erickmu1/Image-Segmentation/blob/main/ImageSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# APS360: Image Segmentation #

### **Team 5** ###
- Bonnie He
- Erick Mejia Uzeda
- Hannah Lee

## Project Description ##

TODO

# Data Loading #

In [ ]:
# Code for loading data

# Data Pre-processing #

In [ ]:
# Code for pre-processing data

# Baseline model: k-means Algorithm #

In [ ]:
# Code for k-means algorithm

# Training Pipeline #

In [ ]:
# Code for the training pipeline

# Model Implementation #

In [ ]:
# Code for implementing the propsed model